In [3]:
import numpy as np
import pandas as pd
import pysam
import pysamstats
#from pysam import VariantFile # for VCF/BCF files
import io
import os
import gzip
from datetime import datetime
import time
import re
import matplotlib.pyplot as plt
import cProfile
import pstats

/Users/Jiazhen/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: pysam.libcalignedsegment.PileupColumn size changed, may indicate binary incompatibility. Expected 72 from C header, got 88 from PyObject
  return f(*args, **kwds)


ModuleNotFoundError: No module named 'matplotlib'

In [2]:
# for gzipped vcf files
def read_vcf(path):
    with gzip.open(path, 'r') as f: 
        lines = [l.decode('UTF-8') for l in f if not l.startswith(b'##')]  # ignore the begining lines 
    #return lines
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})

In [3]:
# for gzipped bed files
def read_bed(path):
    with gzip.open(path, 'r') as f: 
        lines = [l.decode('UTF-8') for l in f if not l.startswith(b'##')]  # ignore the begining lines 
    #return lines
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'CHROM': str, 'START': int, 'END': int},
        sep='\t', header=None,names=['CHROM','START','END']
    )

In [39]:
def assign_label():
    # convert vcf.gz to dataframe
    gatk_in_path = "../gatk-hc.chr20.vcf.gz"
    ture_in_path = "../msb2.b37m.chr20.vcf.gz"
    confi_region_path = "../msb2.b37m.chr20.bed.gz"
    gatk_df = read_vcf(gatk_in_path)
    true_df = read_vcf(ture_in_path)
    
    # 1. assign label
    true_label = np.intersect1d(gatk_df['POS'],true_df['POS']) # in both
    gatk_df['label'] = 0 # false GATK
    gatk_df.loc[gatk_df['POS'].isin(true_label),'label'] = 1 # true variant
    
    # 2. Filter out INDELs
    gatk_df = gatk_df.loc[(gatk_df['REF'].apply(len) == 1) & (gatk_df['ALT'].apply(len) == 1)]
    
    # 3. filter for locations that are only in confident region
    confi_region = read_bed(confi_region_path)
    start_array  = np.array(confi_region['START'])
    end_array = np.array(confi_region['END'])
    keep_index = np.array([]) # initialize empty array

    # keep the index that are in confident region
    for start, end in zip(start_array,end_array):
        # add on location of the pos in confident region
        keep_index = np.append(keep_index,np.where(gatk_df['POS'].between(start,end) == True)[0])
    # keep unique elements
    keep_index = np.unique(keep_index)
    
    gatk_df = gatk_df.iloc[keep_index,:]
    return gatk_df

In [5]:
'''
To create balanced dataset. 
Randomly downsampling true variants 
so that # true var = # false var
'''
def downsample(gatk_df,pos,DEBUG=False):
    if DEBUG:  # selected region of variants
        temp_df = gatk_df.loc[gatk_df['POS'].isin(test_pos)]
        fp_num = sum(temp_df['label'] == 0) # get all fp samples
        bal_tp_pos = temp_df.loc[temp_df['label'] == 1]['POS'].sample(n=fp_num, random_state=1).tolist() # randomly get equal amount of tp samples
        fp_pos = temp_df.loc[temp_df['label'] == 0]['POS'].tolist()
    else: # all variants
        fp_num = sum(gatk_df['label'] == 0) # get all fp samples
        bal_tp_pos = gatk_df.loc[gatk_df['label'] == 1]['POS'].sample(n=fp_num, random_state=1).tolist() 
        fp_pos = gatk_df.loc[gatk_df['label'] == 0]['POS'].tolist() 
    
    return bal_tp_pos + fp_pos

In [53]:
test = np.zeros([2,6])
test[0]

array([0., 0., 0., 0., 0., 0.])

In [130]:
def calc_features(gatk_df, samfile, fastafile, window_size = 10, chrom = '20', DEBUG = False, test_pos = []):
    # for DEBUG
    if(DEBUG): # first two variants
        gatk_pos = downsample(gatk_df,test_pos,DEBUG)
        #label = gatk_df.loc[gatk_df['POS'].isin(gatk_pos),'label']
        #print(label)
    else: # all gatk variants
        gatk_pos = downsample(gatk_df,gatk_df['POS'].tolist(),DEBUG)
        label = gatk_df.loc[gatk_df['POS'].isin(gatk_pos),'label']
        #label = gatk_df['POS'].tolist()
        
    feature_df = pd.DataFrame(columns=['variant_POS','alt_ratio','norm_read_depth','quality_ratio',
                                       'dir_ratio','base_qual', 'ref_encode',
                                       'BaseQRankSum','ClippingRankSum','MQRankSum','label']) 
    # calc average reads depth across the whole chrom ~ take about 4.5 min
    if DEBUG:
        avg_depth = 46  ### for speed up & test only
    else:
        read_stats = pysamstats.load_coverage(samfile, chrom=chrom,pad=True)
        avg_depth = sum(read_stats['reads_all'])/len(read_stats) 
    
    for each_pos in gatk_pos:
        widnow_start = each_pos - int(window_size/2) 
        widnow_end = each_pos + int(window_size/2)
        label = gatk_df.loc[gatk_df['POS'] == each_pos,'label'].values[0]
        
        # Add in hand-crafted features from GATK
        info = re.split(r';+|=',gatk_df.loc[gatk_df['POS']==each_pos,'INFO'].tolist()[0])
        if 'BaseQRankSum' in info:
            bq_sum = info[info.index('BaseQRankSum')+1]
        else:
            bq_sum = 0
        if 'ClippingRankSum' in info:
            cr_sum = info[info.index('ClippingRankSum')+1]
        else:
            cr_sum = 0
        if 'MQRankSum' in info:
            mqr_sum = info[info.index('MQRankSum')+1]
        else:
            mqr_sum = 0
            
        alt_list = []
        norm_depth_list = []
        qual_list = []
        dir_list = []
        base_qual_list = []
        ATGC_array = np.zeros([4,window_size-1]) # ref - probability count initialization
        
        i = 0 # for prob count
        # get reads info via pysam at each var window
        for pileupcolumn in samfile.pileup(chrom, widnow_start, widnow_end, truncate=True,fastafile=fastafile):
            
            ref_base = fastafile.fetch(chrom,pileupcolumn.reference_pos, pileupcolumn.reference_pos+1) # reference base
            map_nt_num = pileupcolumn.get_num_aligned() # number of reads that aligned to the current base position 
            norm_depth = map_nt_num/avg_depth # normalized read depth at current base position 
            norm_depth_list.append(norm_depth)
            
            seq = pileupcolumn.get_query_sequences()
            seq_upper = np.array([elem.upper() for elem in pileupcolumn.get_query_sequences()])
            
            # min Q of {base Q, map Q} -- needs implementation
            quality = np.array(pileupcolumn.get_query_qualities()) # quality of each read here
            
            # encode mutation probability
            prob_qual = 1-10**(np.array(pileupcolumn.get_query_qualities())/-10)
            
            # in case of empty sequence 
            if len(prob_qual) == 0:
                prob_qual = 0
                
            else: # non-empty sequence
                ##### probability based count for ref, 4 x window size - newly added
                ATGC_array[0][i] = sum(prob_qual[seq_upper == 'A'])/len(prob_qual)
                ATGC_array[1][i] = sum(prob_qual[seq_upper == 'T'])/len(prob_qual)
                ATGC_array[2][i] = sum(prob_qual[seq_upper == 'G'])/len(prob_qual)
                ATGC_array[3][i] = sum(prob_qual[seq_upper == 'C'])/len(prob_qual)
                i=i+1
                
                prob_qual = prob_qual[seq_upper != ref_base] # extract mutation 
                # in case of no mutation variant
                if len(prob_qual) == 0:
                    prob_qual = 0
                else:
                    prob_qual = np.mean(prob_qual)
                    
            base_qual_list.append(prob_qual) # newly added feature - probability
            
            qual_ratio = sum(quality[seq_upper != ref_base])/sum(quality) # weighted quality ratio
            qual_list.append(qual_ratio)
            # direction ratio - forward/reverse
            try:
                dir_ratio = sum([nt[0].isupper() if len(nt) > 0 else 0 for nt in seq])/ \
            (sum([nt[0].islower() if len(nt) > 0 else 0 for nt in seq])+ \
             sum([nt[0].isupper() if len(nt) > 0 else 0 for nt in seq]))
                dir_list.append(dir_ratio)
            except Exception as err:
                print("Error:",err)
                print("start",widnow_start)
                print("seq",seq)
                dir_list.append(0)
                
            alt_num = 0 
            
            for pileupread in pileupcolumn.pileups: # check each read for alteratio and other features
                if pileupread.is_del or pileupread.is_refskip: # deletion/insertion? base *
                    alt_num += 1
                    #print('IS DEL',pileupcolumn.reference_pos+1)
                
                if not pileupread.is_del and not pileupread.is_refskip: # SNP case
                    current_nt=pileupread.alignment.query_sequence[pileupread.query_position]
                    if current_nt != ref_base:
                        alt_num += 1
                    
                #if pileupread.is_refskip: # insertion?
                    #print('IS_REFSKIP',pileupcolumn.reference_pos+1)
                    #alt_num += 1
                    
                #if pileupread.indel !=0: # deletion or insertion indel
                    #alt_num += 1
                    
            if map_nt_num == 0:
                alt_ratio = 1 # deletion at the bp location
                
            else:
                alt_ratio = alt_num/map_nt_num
            alt_list.append(alt_ratio)


        feature_df = feature_df.append({'variant_POS':each_pos,'alt_ratio':alt_list,
                                        'norm_read_depth':norm_depth_list,'quality_ratio':qual_list,
                                        'dir_ratio':dir_list,'base_qual':base_qual_list,
                                        'ref_encode':ATGC_array.tolist(),
                                        'BaseQRankSum':bq_sum,
                                        'ClippingRankSum':cr_sum,'MQRankSum':mqr_sum,
                                        'label':label},
                                       ignore_index=True)
        #feature_df['label'] = label # label of each variant candidate
    
    return feature_df


In [71]:
gatk_df = assign_label()

In [5]:
samfile = pysam.AlignmentFile('../msb2.b37m.chr20.bam','rb') 
fastafile = pysam.FastaFile('../chr20.fa')

In [117]:
#test_pos = gatk_df['POS'].sample(frac=0.002, random_state=1).tolist() 
#test_pos = [62899501,2484791]
test_pos = [66370,96596]

In [128]:
trail_df = calc_features(gatk_df, samfile, fastafile, window_size = 33, DEBUG = True, test_pos = test_pos)

In [129]:
trail_df.to_csv("new_data_dec/trail_df.csv",sep='\t')

In [131]:
start = datetime.now()
feature_df = calc_features(gatk_df, samfile, fastafile, window_size = 33, DEBUG = False, test_pos = test_pos)
end = datetime.now()
total_time = end-start
print('Time elapsed (hh:mm:ss.ms) {}'.format(total_time))

/Users/Jiazhen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:88: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


Error: division by zero
start 144883
seq 
Error: division by zero
start 144883
seq 
Error: division by zero
start 12421977
seq 
Error: division by zero
start 12421977
seq 
Error: division by zero
start 51791682
seq 
Error: division by zero
start 38026486
seq 
Error: division by zero
start 38026486
seq 
Error: division by zero
start 60439011
seq 
Error: division by zero
start 2341859
seq 
Error: division by zero
start 2341859
seq 
Error: division by zero
start 2341859
seq 
Error: division by zero
start 5279046
seq 
Error: division by zero
start 5279046
seq 
Error: division by zero
start 5279046
seq 
Error: division by zero
start 5279046
seq 
Error: division by zero
start 5279046
seq 
Error: division by zero
start 5279046
seq 
Error: division by zero
start 5279046
seq 
Error: division by zero
start 5279046
seq 
Error: division by zero
start 6308320
seq 
Error: division by zero
start 6308320
seq 
Error: division by zero
start 6308320
seq 
Error: division by zero
start 6308320
seq 
Error: 

Time elapsed (hh:mm:ss.ms) 0:55:29.048736


In [132]:
samfile.close()
fastafile.close()

#### Split into training, val, test data

In [133]:
def save_result(path,feature_df):
    feature_df.to_csv(path+".csv",sep='\t')
    train_df=feature_df.sample(frac=0.8,random_state=42) # 80% data
    temp_df=feature_df.drop(train_df.index)
    val_df=temp_df.sample(frac=0.5,random_state=42) # 10% data
    test_df=temp_df.drop(val_df.index) # 10% data
    train_path = path+"_train"+".csv"
    val_path = path+"_val"+".csv"
    test_path = path+"_test"+".csv"
    train_df.to_csv(train_path,sep='\t')
    val_df.to_csv(val_path,sep='\t')
    test_df.to_csv(test_path,sep='\t')
    print("Spliting dataset is done")
    
    return train_df['variant_POS'], val_df['variant_POS'], test_df['variant_POS']

In [134]:
train_pos,val_pos,test_pos = save_result("new_data_dec/200204_encode_ref",feature_df)

Spliting dataset is done


#### Re-create 67000 test dataset

### For Debugging

In [53]:
feature_df.loc[feature_df['variant_POS']==69506]['alt_ratio'].tolist()

[[0.034482758620689655,
  0.03571428571428571,
  0.03571428571428571,
  0.038461538461538464,
  0.84,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0]]

In [18]:
gatk_df[np.equal(gatk_df['POS'] > 48280223,gatk_df['POS'] < 48280260)]

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,CHMI_CHMI3_WGS2,label
86495,20,48280229,.,T,A,724.77,.,AC=2;AF=1.00;AN=2;DP=15;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,15:15:51:753,51,0",0
86496,20,48280231,.,T,A,691.77,.,AC=2;AF=1.00;AN=2;DP=16;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,16:16:48:720,48,0",0
86497,20,48280233,.,T,A,691.77,.,AC=2;AF=1.00;AN=2;DP=16;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,16:16:48:720,48,0",0
86498,20,48280235,.,C,A,691.77,.,AC=2;AF=1.00;AN=2;DP=16;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,16:16:48:720,48,0",0
86499,20,48280237,.,T,A,691.77,.,AC=2;AF=1.00;AN=2;DP=16;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,16:16:48:720,48,0",0
86500,20,48280239,.,T,A,691.77,.,AC=2;AF=1.00;AN=2;DP=15;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,15:15:48:720,48,0",0
86501,20,48280241,.,T,A,646.77,.,AC=2;AF=1.00;AN=2;DP=12;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,12:12:45:675,45,0",0
86502,20,48280244,.,A,C,466.77,.,AC=2;AF=1.00;AN=2;DP=11;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,11:11:33:495,33,0",1


In [8]:
chrom='20'
start = 48280239 - 16
end = 48280239 + 16
qual_list = [] 
# each window
#for pileupcolumn in samfile.pileup(chrom, start, end,truncate=True,fastafile=fastafile):
for pileupcolumn in samfile.pileup(chrom, start, end,truncate=True,fastafile=None,compute_baq=False,stepper="samtools"):
    print(pileupcolumn.reference_pos)
    #print("read qual",pileupcolumn.get_query_qualities())
    #print("Prob",1-10**(np.array(pileupcolumn.get_query_qualities())/-10))
    #print(pileupcolumn.nsegments)
    ref_base = fastafile.fetch(chrom,pileupcolumn.reference_pos, pileupcolumn.reference_pos+1)
    print('REF: ', ref_base) # reference base
    #print(pileupcolumn.get_num_aligned())
    #print(len(pileupcolumn.get_query_sequences(mark_matches=True,mark_ends=True,add_indels=False)))
    print("seq:",pileupcolumn.get_query_sequences(mark_matches=True,mark_ends=True,add_indels=True))
    #print("seq 2:",pileupcolumn.get_query_sequences())
    #print('quality: ',pileupcolumn.get_mapping_qualities())
    #print(len(pileupcolumn.get_mapping_qualities()))
    #seq_all = pileupcolumn.get_query_sequences()
    #seq = np.array([elem.upper() for elem in pileupcolumn.get_query_sequences(mark_matches=False,mark_ends=True,add_indels=True)])
    
    #quality = np.array(pileupcolumn.get_mapping_qualities()) # quality of each read here
    #qual_ratio = sum(quality[seq != ref_base])/sum(quality)
    #qual_list.append(qual_ratio)
    #print("quality:",quality)
    #print("qual ratio:",qual_ratio)
    #print(sum([nt[0].isupper() if len(nt) > 0 else '' for nt in seq_all]))
    #dir_ratio = sum([nt[0].isupper() if len(nt) > 0 else '' for nt in seq_all])/ \
    #        (sum([nt[0].islower() if len(nt) > 0 else '' for nt in seq_all])+sum([nt[0].isupper() if len(nt) > 0 else 0 for nt in seq_all]))
    #print("dir_ratio", dir_ratio)
    
    
    # each read
    alt_num=0
    for pileupread in pileupcolumn.pileups: # check each read for alteratio and other features
        ### Clip sum
        aln = pileupread.alignment.cigarstring
        print(aln)
        
        if pileupread.indel < 0: # deletion
            print(pileupread.indel)
        if pileupread.is_del: # deletion
            print(pileupread.is_del)
        if pileupread.is_refskip: # deletion
            print(pileupread.is_refskip)
            #alt_num += 1
        #if not pileupread.is_del and not pileupread.is_refskip: # SNP case?
        #    current_nt=pileupread.alignment.query_sequence[pileupread.query_position]
        #    if current_nt != ref_base:
        #        alt_num += 1
    #print(alt_num)
                    #print("deletion")
    #for pileupread in pileupcolumn.pileups:
    #    if pileupread.is_del and not pileupread.is_refskip and (pileupcolumn.pos >= start ) and (pileupcolumn.pos <= end  ):
    #        print ('%s\t%s\t%s\t%s\t%s\t%s' %
    #             (chrom,
    #              pileupcolumn.pos +1,                                                     # +1 needed because pysam is 0-based and most programs (and human heads) are 1-based
    #              fastafile.fetch(chrom,pileupcolumn.reference_pos ,pileupcolumn.reference_pos +1), # ref base
    #              pileupread.alignment.query_name,
    #              pileupread.query_position,  
    #              "*"))     
    #    if not pileupread.is_del and not pileupread.is_refskip and (pileupcolumn.pos >= start ) and (pileupcolumn.pos <= end  ):
    #        print ('%s\t%s\t%s\t%s\t%s\t%s' %
    #             (chrom,
    #              pileupcolumn.pos +1,
    #              fastafile.fetch(chrom,pileupcolumn.reference_pos ,pileupcolumn.reference_pos +1),  
    #              pileupread.alignment.query_name,
    #              pileupread.query_position,  
    #              pileupread.alignment.query_sequence[pileupread.query_position]))

48280223
REF:  A
seq: ['a$', 'A', 'a', 'a', 'A', 'A', 'A', 'A', 'a', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'a', 'A']
148M1I2M
147M1I3M
147M1I3M
145M1I5M
134M1I5M11S
128M1I5M17S
104M1I5M41S
91M1I5M54S
58S75M1I5M12S
72M1I5M73S
49M1I101M
39M2I100M10S
37M1I5M108S
38M113S
28M1I122M
6S19M126S
126S19M6S
11S38M102S
48280224
REF:  A
seq: ['A$', 'a$', 'a', 'A', 'A', 'A', 'A', 'a', 'A', 'A', 'A', 'A', 'A', 'a', 'A']
147M1I3M
147M1I3M
145M1I5M
134M1I5M11S
128M1I5M17S
104M1I5M41S
91M1I5M54S
58S75M1I5M12S
72M1I5M73S
49M1I101M
39M2I100M10S
28M1I122M
6S19M126S
126S19M6S
11S38M102S
48280225
REF:  A
seq: ['a', 'A', 'A', 'A', 'A', 'A', 'a', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'a', 'A']
145M1I5M
134M1I5M11S
128M1I5M17S
104M1I5M41S
91M1I5M54S
75M1I5M70S
58S75M1I5M12S
72M1I5M73S
49M1I101M
39M2I100M10S
37M1I5M108S
38M113S
28M1I122M
26M1I5M119S
6S19M126S
126S19M6S
11S38M102S
48280226
REF:  A
seq: ['a$', 'A$', 'A$', 'A$', 'A$', 'a$', 'A$', 'A', 'A', 'A$', 'A', 'A', 'A$', 'A', 'a', 'A']
145M1I5M
134M1I5M11S
12

## Different dimensions 
for i,j in zip(feature_df['alt_ratio'],feature_df['variant_POS']):
    if len(i) != 10:
        print(j)

In [52]:
len(feature_df.loc[feature_df['variant_POS'] ==51170718]['alt_ratio'])

1

In [6]:
plt.hist(feature_df['BaseQRankSum'].tolist(),alpha=0.5,label = 'all_variants')
#plt.hist(true_pos_dist,alpha=0.5, label = 'TP')
plt.xlabel("Chrom position",fontSize=15)
plt.ylabel("Count",fontSize=15)
plt.title("Location of all variant",fontSize=15)
plt.legend()

NameError: name 'feature_df' is not defined

In [102]:
feature_df.loc[feature_df['variant_POS'] ==54423713]

,variant_POS,alt_ratio,norm_read_depth,quality_ratio,dir_ratio,base_qual,BaseQRankSum,ClippingRankSum,MQRankSum,label
3900,54423713,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03125, 0...","[0.7503736779910214, 0.7503736779910214, 0.728...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01414581...","[0.5142857142857142, 0.4857142857142857, 0.529...","[0, 0, 0, 0, 0, 0, 0, 0.9498812766372727, 0, 0...",1.151,-0.516,-4.523,1


In [32]:
pd.read_csv("profile/test_200120.txt")

ParserError: Error tokenizing data. C error: Expected 1 fields in line 19, saw 2
